## This notebook explores 2 runway records that were kicked back from Lex's R code. 
### The output of this notebook is:
#### 1. 2 Lat/Long calculated for Runways

In [1]:
# Import all needed libraries
import pandas as pd
import numpy as np
import geopy
from geopy.distance import geodesic
import re

In [2]:
# Read in csv file
nonstd_rwy = pd.read_csv('C:/Users/Olga/Desktop/GMU/DAEN690/LEGO/Olga - All_Points_Non-Standard_Runways_2OUTOF2.csv')
nonstd_rwy

,REMARKS,UAS_LOC,RUNWAY,DIRECTION,AIRPORT,dateonly,timeonly,DATE,CALLSIGN,POD,...,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,DATASET,UASLOCATION,RWYLOCATION,cityname
0,uas:Fairfax County Police reported sighting a ...,1 NM E of NCRCC,['rwy 1r'],[],"['IAD', 'DCA', 'CRC', 'IAD', 'TCT']",2/4/2019,22:05,2019-02-04T22:05Z,NaN,NCRCC,...,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY,1 NM E of IAD,RWY 01R,NaN
1,Received a report via telephone of a uas sight...,2 NM S of NCRCC,['rwy 1r'],[],"['IAD', 'CRC', 'IAD', 'TCT', 'IAD']",1/29/2019,4:35,2019-01-29T04:35Z,NaN,NCRCC,...,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY,1 NM E of IAD,RWY 01R,NaN


In [3]:
# Extract IDENTs from UAS location column
uas_loc = nonstd_rwy['UASLOCATION'].tolist()
uas_rwy = []

for i in range(len(uas_loc)):
    rwy = uas_loc[i].split(' ')[-1]
    
    if len(rwy) <= 4:
        uas_rwy.append(rwy)
    else:
        trim_air = rwy[-3:]
        uas_rwy.append(trim_air)
        
uas_rwy_df = pd.DataFrame()
uas_rwy_df['IDENT'] = uas_rwy
uas_rwy_df

,IDENT
0,IAD
1,IAD


In [4]:
# Add needed columns to the IDENTs dataframe
add_col = nonstd_rwy['REMARKS'].tolist()
uas_rwy_df['REMARKS'] = add_col
add_col1 = nonstd_rwy['UASLOCATION'].tolist()
uas_rwy_df['UASLOCATION'] = add_col1
add_col1 = nonstd_rwy['RWYLOCATION'].tolist()
uas_rwy_df['RWYLOCATION'] = add_col1
uas_rwy_df

,IDENT,REMARKS,UASLOCATION,RWYLOCATION
0,IAD,uas:Fairfax County Police reported sighting a ...,1 NM E of IAD,RWY 01R
1,IAD,Received a report via telephone of a uas sight...,1 NM E of IAD,RWY 01R


In [5]:
# Creating dictionary for all bearing abbreviations and their degrees
bearing_deg = {
    'N' : 0,
    'NNE' : 23,
    'NE' : 45,
    'ENE' : 68,
    'E' : 90,
    'ESE' : 113,
    'SE' : 135,
    'SSE' : 158,
    'S' : 180,
    'SSW' : 203,
    'SW' : 225,
    'WSW' : 248,
    'W' : 270,
    'WNW' : 293,
    'NW' : 315,
    'NNW' : 338
}

# Separate Distances and bearings into two different columns
uas_loc = uas_rwy_df['UASLOCATION']
distances = []
bearings = []

for i in range(len(uas_loc)):
    split = uas_loc[i].split(' ')[0:3]
    
    if 'NM' in split[0]: 
        distance_nm = split[0][0:len(split[0]) - 2]
        bearing_abrev = split[1].replace('of', '')
        
    else: 
        distance_nm = split[0]
        bearing_abrev = split[2].replace('of', '')
    
    distances.append(distance_nm[0:])
    bearings.append(bearing_abrev)

# Create new dataframe for UAS locations
new_df = pd.DataFrame(uas_rwy_df['REMARKS'])
new_df['UAS_LOC'] = uas_rwy_df['UASLOCATION']
new_df['IDENT'] = uas_rwy_df['IDENT']
new_df['Distance'] = distances
new_df['Bearing'] = bearings
new_df['RWYLOCATION'] = uas_rwy_df['RWYLOCATION']

new_df

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,RWYLOCATION
0,uas:Fairfax County Police reported sighting a ...,1 NM E of IAD,IAD,1,E,RWY 01R
1,Received a report via telephone of a uas sight...,1 NM E of IAD,IAD,1,E,RWY 01R


In [6]:
# Read in the linked airports and runways dataset
airport_runway = pd.read_csv('C:/Users/Olga/Desktop/GMU/DAEN690/LEGO/Airports&Runways.csv')
airport_runway

,DESIGNATOR,RUNWAY.LONGITUDE,RUNWAY.LATITUDE,IDENT,ICAO_ID,NAME,LATITUDE,LONGITUDE
0,5/23,-176.645456,51.882484,ADK,PADK,Adak,51.883583,-176.642482
1,4/22,-154.181625,56.938950,AKK,PAKH,Akhiok,56.938691,-154.182556
2,1/19,-161.492556,60.914625,Z13,PFZK,Akiachak,60.913810,-161.493329
3,E/W,-0.199980,-0.000081,KKI,NaN,Akiachak,60.907865,-161.435077
4,NW/SE,-0.199980,-0.000081,KKI,NaN,Akiachak,60.907865,-161.435077
...,...,...,...,...,...,...,...,...
24031,176/356,-99.265570,34.676719,LTS,KLTS,Altus AFB,34.667975,-99.267750
24032,18L/36R,-99.259598,34.667652,LTS,KLTS,Altus AFB,34.667975,-99.267750
24033,18R/36L,-99.273881,34.660327,LTS,KLTS,Altus AFB,34.667975,-99.267750
24034,1/19,-84.946625,36.350671,2A1,NaN,Jamestown Muni,36.349694,-84.946639


In [7]:
# Merge the dataset with the runway locations dataframe
uas_rwy_loc= pd.merge(new_df, airport_runway, on='IDENT', how='left')
uas_rwy_loc

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,RWYLOCATION,DESIGNATOR,RUNWAY.LONGITUDE,RUNWAY.LATITUDE,ICAO_ID,NAME,LATITUDE,LONGITUDE
0,uas:Fairfax County Police reported sighting a ...,1 NM E of IAD,IAD,1,E,RWY 01R,01C/19C,-77.459557,38.958019,KIAD,Washington Dulles Intl,38.947444,-77.459944
1,uas:Fairfax County Police reported sighting a ...,1 NM E of IAD,IAD,1,E,RWY 01R,01L/19R,-77.474643,38.960459,KIAD,Washington Dulles Intl,38.947444,-77.459944
2,uas:Fairfax County Police reported sighting a ...,1 NM E of IAD,IAD,1,E,RWY 01R,01R/19L,-77.436212,38.936393,KIAD,Washington Dulles Intl,38.947444,-77.459944
3,uas:Fairfax County Police reported sighting a ...,1 NM E of IAD,IAD,1,E,RWY 01R,12/30,-77.469700,38.937721,KIAD,Washington Dulles Intl,38.947444,-77.459944
4,Received a report via telephone of a uas sight...,1 NM E of IAD,IAD,1,E,RWY 01R,01C/19C,-77.459557,38.958019,KIAD,Washington Dulles Intl,38.947444,-77.459944
5,Received a report via telephone of a uas sight...,1 NM E of IAD,IAD,1,E,RWY 01R,01L/19R,-77.474643,38.960459,KIAD,Washington Dulles Intl,38.947444,-77.459944
6,Received a report via telephone of a uas sight...,1 NM E of IAD,IAD,1,E,RWY 01R,01R/19L,-77.436212,38.936393,KIAD,Washington Dulles Intl,38.947444,-77.459944
7,Received a report via telephone of a uas sight...,1 NM E of IAD,IAD,1,E,RWY 01R,12/30,-77.469700,38.937721,KIAD,Washington Dulles Intl,38.947444,-77.459944


In [8]:
# Split the designator for further use
designator = []

for i in range(len(uas_rwy_loc)):
    split = uas_rwy_loc['RWYLOCATION'][i].split(' ')
    designator.append(split[1])
    
uas_rwy_loc['RWY_DESIGNATOR'] = designator
uas_rwy_loc

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,RWYLOCATION,DESIGNATOR,RUNWAY.LONGITUDE,RUNWAY.LATITUDE,ICAO_ID,NAME,LATITUDE,LONGITUDE,RWY_DESIGNATOR
0,uas:Fairfax County Police reported sighting a ...,1 NM E of IAD,IAD,1,E,RWY 01R,01C/19C,-77.459557,38.958019,KIAD,Washington Dulles Intl,38.947444,-77.459944,01R
1,uas:Fairfax County Police reported sighting a ...,1 NM E of IAD,IAD,1,E,RWY 01R,01L/19R,-77.474643,38.960459,KIAD,Washington Dulles Intl,38.947444,-77.459944,01R
2,uas:Fairfax County Police reported sighting a ...,1 NM E of IAD,IAD,1,E,RWY 01R,01R/19L,-77.436212,38.936393,KIAD,Washington Dulles Intl,38.947444,-77.459944,01R
3,uas:Fairfax County Police reported sighting a ...,1 NM E of IAD,IAD,1,E,RWY 01R,12/30,-77.469700,38.937721,KIAD,Washington Dulles Intl,38.947444,-77.459944,01R
4,Received a report via telephone of a uas sight...,1 NM E of IAD,IAD,1,E,RWY 01R,01C/19C,-77.459557,38.958019,KIAD,Washington Dulles Intl,38.947444,-77.459944,01R
5,Received a report via telephone of a uas sight...,1 NM E of IAD,IAD,1,E,RWY 01R,01L/19R,-77.474643,38.960459,KIAD,Washington Dulles Intl,38.947444,-77.459944,01R
6,Received a report via telephone of a uas sight...,1 NM E of IAD,IAD,1,E,RWY 01R,01R/19L,-77.436212,38.936393,KIAD,Washington Dulles Intl,38.947444,-77.459944,01R
7,Received a report via telephone of a uas sight...,1 NM E of IAD,IAD,1,E,RWY 01R,12/30,-77.469700,38.937721,KIAD,Washington Dulles Intl,38.947444,-77.459944,01R


In [9]:
# Add a new column to verify if the runway belongs to the airport (TRUE) or not (FALSE)
uas_rwy_loc['True/False'] = uas_rwy_loc.apply(lambda i: str(i.RWY_DESIGNATOR) in str(i.DESIGNATOR), axis=1)
uas_rwy_loc

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,RWYLOCATION,DESIGNATOR,RUNWAY.LONGITUDE,RUNWAY.LATITUDE,ICAO_ID,NAME,LATITUDE,LONGITUDE,RWY_DESIGNATOR,True/False
0,uas:Fairfax County Police reported sighting a ...,1 NM E of IAD,IAD,1,E,RWY 01R,01C/19C,-77.459557,38.958019,KIAD,Washington Dulles Intl,38.947444,-77.459944,01R,False
1,uas:Fairfax County Police reported sighting a ...,1 NM E of IAD,IAD,1,E,RWY 01R,01L/19R,-77.474643,38.960459,KIAD,Washington Dulles Intl,38.947444,-77.459944,01R,False
2,uas:Fairfax County Police reported sighting a ...,1 NM E of IAD,IAD,1,E,RWY 01R,01R/19L,-77.436212,38.936393,KIAD,Washington Dulles Intl,38.947444,-77.459944,01R,True
3,uas:Fairfax County Police reported sighting a ...,1 NM E of IAD,IAD,1,E,RWY 01R,12/30,-77.469700,38.937721,KIAD,Washington Dulles Intl,38.947444,-77.459944,01R,False
4,Received a report via telephone of a uas sight...,1 NM E of IAD,IAD,1,E,RWY 01R,01C/19C,-77.459557,38.958019,KIAD,Washington Dulles Intl,38.947444,-77.459944,01R,False
5,Received a report via telephone of a uas sight...,1 NM E of IAD,IAD,1,E,RWY 01R,01L/19R,-77.474643,38.960459,KIAD,Washington Dulles Intl,38.947444,-77.459944,01R,False
6,Received a report via telephone of a uas sight...,1 NM E of IAD,IAD,1,E,RWY 01R,01R/19L,-77.436212,38.936393,KIAD,Washington Dulles Intl,38.947444,-77.459944,01R,True
7,Received a report via telephone of a uas sight...,1 NM E of IAD,IAD,1,E,RWY 01R,12/30,-77.469700,38.937721,KIAD,Washington Dulles Intl,38.947444,-77.459944,01R,False


In [10]:
# Find all the records where the runways belong to the corresponding airport
uas_rwy_loc = uas_rwy_loc[uas_rwy_loc['True/False'] == True]
uas_rwy_loc

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,RWYLOCATION,DESIGNATOR,RUNWAY.LONGITUDE,RUNWAY.LATITUDE,ICAO_ID,NAME,LATITUDE,LONGITUDE,RWY_DESIGNATOR,True/False
2,uas:Fairfax County Police reported sighting a ...,1 NM E of IAD,IAD,1,E,RWY 01R,01R/19L,-77.436212,38.936393,KIAD,Washington Dulles Intl,38.947444,-77.459944,01R,True
6,Received a report via telephone of a uas sight...,1 NM E of IAD,IAD,1,E,RWY 01R,01R/19L,-77.436212,38.936393,KIAD,Washington Dulles Intl,38.947444,-77.459944,01R,True


In [11]:
# Export all records to csv file
uas_rwy_loc.to_csv('RWY2.csv', index = False)
uas_rwy_loc = pd.read_csv('C:/Users/Olga/Desktop/GMU/DAEN690/LEGO/RWY2.csv')

In [12]:
# Replace designator with respective degrees
designator = uas_rwy_loc['RWY_DESIGNATOR']
bearing = []

for i in range(len(uas_rwy_loc)):
    bearing.append(int(re.sub("\D","",designator[i]))*10)
    
uas_rwy_loc['RWY_BEARING'] = bearing
uas_rwy_loc

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,RWYLOCATION,DESIGNATOR,RUNWAY.LONGITUDE,RUNWAY.LATITUDE,ICAO_ID,NAME,LATITUDE,LONGITUDE,RWY_DESIGNATOR,True/False,RWY_BEARING
0,uas:Fairfax County Police reported sighting a ...,1 NM E of IAD,IAD,1,E,RWY 01R,01R/19L,-77.436212,38.936393,KIAD,Washington Dulles Intl,38.947444,-77.459944,01R,True,10
1,Received a report via telephone of a uas sight...,1 NM E of IAD,IAD,1,E,RWY 01R,01R/19L,-77.436212,38.936393,KIAD,Washington Dulles Intl,38.947444,-77.459944,01R,True,10


In [13]:
# Converting distances to kilometers and append it to dataframe
uas_rwy_loc['Distance'] = pd.to_numeric(uas_rwy_loc['Distance'],errors = 'coerce')
dist_kilo = []

for i in range(len(uas_rwy_loc)):
    distanceKilo = uas_rwy_loc['Distance'][i] * 1.852 
    dist_kilo.append(distanceKilo)
    
uas_rwy_loc['Distance_Kilometers'] = dist_kilo
uas_rwy_loc

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,RWYLOCATION,DESIGNATOR,RUNWAY.LONGITUDE,RUNWAY.LATITUDE,ICAO_ID,NAME,LATITUDE,LONGITUDE,RWY_DESIGNATOR,True/False,RWY_BEARING,Distance_Kilometers
0,uas:Fairfax County Police reported sighting a ...,1 NM E of IAD,IAD,1,E,RWY 01R,01R/19L,-77.436212,38.936393,KIAD,Washington Dulles Intl,38.947444,-77.459944,01R,True,10,1.852
1,Received a report via telephone of a uas sight...,1 NM E of IAD,IAD,1,E,RWY 01R,01R/19L,-77.436212,38.936393,KIAD,Washington Dulles Intl,38.947444,-77.459944,01R,True,10,1.852


In [14]:
# Calculate UAS lat/long coordinates using geopy
uas_lat = []
uas_long = []


for i in range(len(uas_rwy_loc)):
    lat_rwy = pd.to_numeric(uas_rwy_loc['RUNWAY.LATITUDE'][i])
    long_rwy = pd.to_numeric(uas_rwy_loc['RUNWAY.LONGITUDE'][i])
    b = pd.to_numeric(uas_rwy_loc['RWY_BEARING'][i])
    d = pd.to_numeric(uas_rwy_loc['Distance_Kilometers'][i])

    origin = geopy.Point(lat_rwy, long_rwy)
    destination = geodesic(kilometers=d).destination(origin,b)

    lat2, lon2, = destination.latitude, destination.longitude

    uas_lat.append(lat2)
    uas_long.append(lon2)

# Append UAS Lat/Long information to DataFrame
uas_rwy_loc['UAS_Latitude'] = uas_lat
uas_rwy_loc['UAS_Longitude'] = uas_long
uas_rwy_loc

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,RWYLOCATION,DESIGNATOR,RUNWAY.LONGITUDE,RUNWAY.LATITUDE,ICAO_ID,NAME,LATITUDE,LONGITUDE,RWY_DESIGNATOR,True/False,RWY_BEARING,Distance_Kilometers,UAS_Latitude,UAS_Longitude
0,uas:Fairfax County Police reported sighting a ...,1 NM E of IAD,IAD,1,E,RWY 01R,01R/19L,-77.436212,38.936393,KIAD,Washington Dulles Intl,38.947444,-77.459944,01R,True,10,1.852,38.952822,-77.432502
1,Received a report via telephone of a uas sight...,1 NM E of IAD,IAD,1,E,RWY 01R,01R/19L,-77.436212,38.936393,KIAD,Washington Dulles Intl,38.947444,-77.459944,01R,True,10,1.852,38.952822,-77.432502


In [15]:
# Export dataframe to csv file
uas_rwy_loc = uas_rwy_loc.drop(['Distance','Bearing','Distance_Kilometers','RWY_BEARING','ICAO_ID','RWYLOCATION', 'DESIGNATOR','RWY_DESIGNATOR','True/False','LATITUDE','LONGITUDE'], 1)
uas_rwy_loc = uas_rwy_loc.rename({'RUNWAY.LONGITUDE': 'Origin_Longitude', 'RUNWAY.LATITUDE': 'Origin_Latitude'}, axis=1)
uas_rwy_loc = pd.merge(uas_rwy_loc, nonstd_rwy, on='REMARKS', how='left')
uas_rwy_loc.to_csv('COMPLETED-Non-std-2Runways.csv', index = False)

In [ ]:
##########################################################################################################################